In [1]:
import numpy as np 
import pandas as pd 
from ipynb.fs.full.funciones_FF import feature_RatioDayMoments

# Seteo del mes snapshot

In [2]:
mes_snapshot = 9  #9 para train 12 para scoreo

# Lectura de los csv

In [3]:
df_pageviews = pd.read_csv("./data/pageviews.csv",
                   parse_dates=["FEC_EVENT"])
df_devicedata = pd.read_csv("./data/device_data.csv",
                   parse_dates=["FEC_EVENT"])
df_conversiones = pd.read_csv("./data/conversiones.csv")

In [4]:
#Se eliminan los registros que no deben ser considerados.
df_pageviews = df_pageviews[df_pageviews['FEC_EVENT'].dt.month <= mes_snapshot]
df_pageviews = df_pageviews[df_pageviews['FEC_EVENT'].dt.month > mes_snapshot - 9]

df_devicedata = df_devicedata[df_devicedata['FEC_EVENT'].dt.month <= mes_snapshot]
df_devicedata = df_devicedata[df_devicedata['FEC_EVENT'].dt.month > mes_snapshot - 9]

# Creación del universo

In [10]:
def universo_train(df):
    universo = pd.DataFrame()
    universo['user_id'] = df.USER_ID.unique()
    universo['snapshot_mes'] = mes_snapshot
    return universo

In [21]:
universo = universo_train(df_pageviews)

# Features

In [ ]:
def feature_X(df):
    

In [ ]:
def feature_X1(df):

# Armar dataset

In [22]:
universo = feature_RatioDayMoments(universo)
universo

,user_id,snapshot_mes,FEC_EVENT_MAX,CANT_TOTAL_EVER,CANT_TOTAL_ULT_7D,FEC_EVENT_MENOS_7D,RATIO_VISITA_MADRUGADA,RATIO_VISITA_MANANA,RATIO_VISITA_TARDE,RATIO_VISITA_NOCHE
0,0,9,2018-12-31 09:56:25,6431,132,2018-12-24 09:56:25,0.000000,0.545455,0.454545,0.000000
1,1,9,2018-12-30 23:29:46,2111,27,2018-12-23 23:29:46,0.000000,0.333333,0.407407,0.259259
2,2,9,2018-12-27 18:56:29,2368,20,2018-12-20 18:56:29,0.000000,0.000000,0.050000,0.950000
3,3,9,2018-12-30 13:50:16,1935,85,2018-12-23 13:50:16,0.000000,0.070588,0.600000,0.329412
4,4,9,2018-12-29 01:02:38,3105,40,2018-12-22 01:02:38,0.025000,0.625000,0.350000,0.000000
5,5,9,2018-12-31 20:25:58,5878,134,2018-12-24 20:25:58,0.000000,0.432836,0.455224,0.111940
6,6,9,2018-12-26 09:35:33,1642,52,2018-12-19 09:35:33,0.000000,0.557692,0.442308,0.000000
7,7,9,2018-12-27 12:39:56,2562,34,2018-12-20 12:39:56,0.000000,0.117647,0.882353,0.000000
8,8,9,2018-12-27 11:36:53,2299,54,2018-12-20 11:36:53,0.000000,0.074074,0.907407,0.018519
9,9,9,2018-12-31 17:48:44,2614,111,2018-12-24 17:48:44,0.000000,0.531532,0.243243,0.225225


# Target

In [23]:
def target(df):
    df = pd.merge(df, df_conversiones, how='left', left_on=['user_id'], right_on=['USER_ID'])
    df.mes.fillna(value= 99, inplace=True)
    df = df.astype({'mes': 'int32'}) 
    
    df['target'] = 0
    df.loc[df.mes == df.snapshot_mes +1, 'target'] = 1
    df.loc[df.mes == df.snapshot_mes +2, 'target'] = 1
    df.loc[df.mes == df.snapshot_mes +3, 'target'] = 1

    return (df.groupby(['user_id', 'snapshot_mes'])['target'].max()).to_frame().reset_index()

In [25]:
target_universo = target(universo)
universo = pd.merge(universo, target_universo, how='left', left_on=['user_id', 'snapshot_mes'], right_on=['user_id','snapshot_mes'])

,user_id,snapshot_mes,FEC_EVENT_MAX,CANT_TOTAL_EVER,CANT_TOTAL_ULT_7D,FEC_EVENT_MENOS_7D,RATIO_VISITA_MADRUGADA,RATIO_VISITA_MANANA,RATIO_VISITA_TARDE,RATIO_VISITA_NOCHE,target
0,0,9,2018-12-31 09:56:25,6431,132,2018-12-24 09:56:25,0.000000,0.545455,0.454545,0.000000,0
1,1,9,2018-12-30 23:29:46,2111,27,2018-12-23 23:29:46,0.000000,0.333333,0.407407,0.259259,0
2,2,9,2018-12-27 18:56:29,2368,20,2018-12-20 18:56:29,0.000000,0.000000,0.050000,0.950000,0
3,3,9,2018-12-30 13:50:16,1935,85,2018-12-23 13:50:16,0.000000,0.070588,0.600000,0.329412,0
4,4,9,2018-12-29 01:02:38,3105,40,2018-12-22 01:02:38,0.025000,0.625000,0.350000,0.000000,0
5,5,9,2018-12-31 20:25:58,5878,134,2018-12-24 20:25:58,0.000000,0.432836,0.455224,0.111940,0
6,6,9,2018-12-26 09:35:33,1642,52,2018-12-19 09:35:33,0.000000,0.557692,0.442308,0.000000,0
7,7,9,2018-12-27 12:39:56,2562,34,2018-12-20 12:39:56,0.000000,0.117647,0.882353,0.000000,0
8,8,9,2018-12-27 11:36:53,2299,54,2018-12-20 11:36:53,0.000000,0.074074,0.907407,0.018519,0
9,9,9,2018-12-31 17:48:44,2614,111,2018-12-24 17:48:44,0.000000,0.531532,0.243243,0.225225,0


In [33]:
universo.to_csv(r'dataset.csv', index = None, header=True) 